In [24]:
import numpy as np
import scipy as sp
import random
from operator import itemgetter
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation

In [34]:
#Parameters

a = 1.               # average bird radius
width_a = a / 10.    # width of the gaussian distribution

N_xy = 10            # Number of penguins in the x and  y direction
N_z = 10             # Number of penguins in the z direction

N = 10

# Physical Constants
F_self = 6.0
F_in = 5.0
k = -2.0

T_in = -2.0
T_n = 0.1
T_align = 0.1

mass = 1.0

In [50]:
data = []

class Birds(object):
    
    def __init__(self, r, pos, coordinate):
        self.r = r         # radius of the bird (bird looks like a disk)
        self.pos = pos     # bird position (x,y,z)
        self.coordinate = coordinate     # unit vector
        self.boundary = False
        self.bisection_angle = 0
        self.bisector = np.zeros(2)
        
    def __str__(self):
        return "Bird located at ([], [])".format(self.pos[0], self.pos[1])
    
    def __repr_(self):
        return self.__str__()
    
    def distance(self, bird2):
        return self.pos - bird2.pos
    
    def update_pos(self, pos):
        self.pos = pos
        
    def update_coord(self, coordinate):
        self.coordinate = coordinate
        
    def find_exterior_bisector(self, bird_list,a):
        r_crit = 1.5 * a
        
        r_theta_list = []
        
        for i in range(len(bird_list)):
            
            if (bird_list[i] != self):
                r = self.distance(bird_list[i])
                r_mag = np.linalg.norm(r)
                
                if (r_mag < r_crit):
                    theta = np.arctan2(r[1], r[0])
                    r_theta_list.append([r, theta])
        
        r_theta_list = sorted(r_theta_list, key=itemgetter(1))
        
        for j in range(len(r_theta_list)):
            
            try:
                a = j
                b = j + 1
                diff = r_theta_list[b][1] = r_theta_list[a][1]
                
            except IndexError:
                a = 0
                b = j
                diff = (np.pi - r_theta_list[b][1]) + (r_theta_list[a][1] + np.pi)
            
            if (diff >= np.pi):
                self.boundary = True
                self.bisection_angle = diff
                
                exterior_bisector = -1.0 * (r_theta_list[a][0] + r_theta_list[b][0])
                
                angle1 = np.arctan2(r_theta_list[a][0][1], r_theta_list[a][0][0]) * 180 / np.pi
                angle2 = np.arctan2(r_theta_list[b][0][1], r_theta_list[b][0][0]) * 180 / np.pi
                
                if (exterior_bisector[0] == [0]) and (exterior_bisector[1] == 0):
                    if (angle1 == 0) and (angle2 == 180):
                        exterior_bisector = np.array([0.0, -1,0])
                    if (angle1 == 180) and (angle2 == 0):
                        exterior_bisector = np.array([0.0, 1,0])
                    if (angle1 == 90) and (angle2 == -90):
                        exterior_bisector = np.array([-1.0, 0,0])
                    if (angle1 == -90) and (angle2 == 90):
                        exterior_bisector = np.array([1.0, 0,0])
                        
                self.bisector = exterior_bisector
                
                break
            
            else:
                self.boundary = False
                 
    def force(self, F_self, F_in, k, bird_list, a):
        # The net force on the particle
        # F_self & F_in set the strength of the interactions
        # k is the spring constant and represents the strength of the repulsion force
        # bird_list is a list of all the birds in the system
        # a is radius of the birds
        
        r_crit = 1.3 * a
        
        F_selfpropulsion = F_self * self.coordinate
        F_repulsion = np.zeros(2)
        
        for i in range(len(bird_list)):
            
            if (bird_list[i] != self):
                if(self.boundary == True):
                    F_boundary = F_in * self.coordinate * (self.bisection_angle - np.pi)
                else:
                    F_boundary = 0
                    
                r = self.distance(bird_list[i])
                r_mag = np.linalg.norm(r)
                
                if (r_mag < r_crit):
                    F_repulsion += -k * r
                
        return F_boundary + F_repulsion + F_selfpropulsion

    def torque(self, T_in, T_n, T_align, bird_list, a):
        # The net torque on the particle
        # T_in is a strength parameter for the boundary torque
        # T_n is a strength parameter for the noise torque
        # T_align is a stregth parameter for the alignment torque
        # bird_list is a list containing all the birds in the system
        # a is the average radius of all the birds in the system
        
        r_crit = 1.3 * a
        
        if (self.boundary == True):
            T_boundary = T_in * (self.coordinate - self.bisector)
        else:
            T_boundary = 0
            
        zeta = np.random.uniform(-1.0,1.0)
        
        T_random = T_n * zeta
        
        T_alignment = np.zeros(2)
        
        for i in range(len(bird_list)):                
            if (bird_list[i] != self):
                
                r = self.distance(bird_list[i])
                r_mag = np.linalg.norm(r)
                
                if (r_mag < r_crit):
                    T_alignment += T_align * (self.coordinate - bird_list[i].coordinate)
        
        return T_boundary + T_random + T_alignment
    
# Plot

def bird_plot(bird_list, a):
    for birds in bird_list:
        birds.find_exterior_bisector(bird_list, a)
        
        if (birds.boundary == True):
            plt.plot(birds.pos[0], birds.pos[1], "ro")
        else:
            plt.plot(birds.pos[1], bird.pos[1], "bo")
    
    plt.show()


# Initialize  

def birds_init(N, a):
    bird_list = []
    
    for i in range(N):
        for j in range(N):
            x = i + np.random.uniform(-0.2, 0.2)
            y = j + np.random.uniform(-0.2, 0.2)
            
            pos = np.array([x,y])
            
            x_coord = 0.01 + np.random.uniform(-0.005, 0.005)
            y_coord = 0
            
            coordinate = np.array([x_coord, y_coord])
            
            r = a
            
            birds = Birds(r, pos, coordinate)
            bird_list.append(birds)
    return bird_list

def init():
    global ax
    path = ax.scatter([], [])
    return path

# Graph

def update_graph(i):
    ax.clear()
    
    ax.set_ylim([-5, 15])
    ax.set_ylabel("Y")
    
    ax.set_xlim([-5, 15])
    ax.set_xlabel("X")
    
    x = data[0][i]
    y = data[1][i]
    
    path = ax.scatter(x, y)
    
    return path

# Run

def move_birds(bird_list, a):
    t_start = 0.01
    t_end = 1.5
    t_iter = 0.01
    
    global data
    Xs = []
    Ys = []
    
    while t_start < t_end:
        Xs_timestep = []
        Ys_timestep = []
        
        for i in range(len(bird_list)):
            birds = bird_list[i]
            force = birds.force(F_self, F_in, k, bird_list, a)
            torque = birds.torque(T_in, T_n, T_align, bird_list,a)
            
            I = (2.0/5.0) * mass * birds.r**2.0 # moment of inertia
            alpha = torque / I
            
            acc = force / mass # acceleration
            
            new_pos = birds.pos + acc * t_iter
            new_coord = birds.coordinate + alpha * t_iter
            
            birds.update_pos(new_pos)
            birds.update_coord(new_coord)
            
            Xs_timestep.append(new_pos[0])
            Ys_timestep.append(new_pos[1])
            
            birds.find_exterior_bisector(bird_list, a)
            
        Xs.append(Xs_timestep)
        Ys.append(Ys_timestep)
        #print ("---- time={t} ({t_start}/{n}) ----".format(

        print(round(t_start,2))
        t_start += t_iter
    
    data.append(Xs)
    data.append(Ys)
    

# Main

def main_birds():
     
        frames = 149
        
        a = 1.0
        
        bird_list = birds_init(N,a)
        
        move_birds(bird_list, a)
        
        fig = plt.figure()
        global ax
        ax = fig.add_subplot(111)
        
        anim = animation.FuncAnimation(fig, update_graph, frames, blit=False, init_func=init)

        plt.show()

main_birds()

0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.69
0.7
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.82
0.83
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.94
0.95
0.96
0.97
0.98
0.99
1.0
1.01
1.02
1.03
1.04
1.05
1.06
1.07
1.08
1.09
1.1
1.11
1.12
1.13
1.14
1.15
1.16
1.17
1.18
1.19
1.2
1.21
1.22
1.23
1.24
1.25
1.26
1.27
1.28
1.29
1.3
1.31
1.32
1.33
1.34
1.35
1.36
1.37
1.38
1.39
1.4
1.41
1.42
1.43
1.44
1.45
1.46
1.47
1.48
1.49
